In [1]:
import sys
import os

sys.path.insert(0, os.path.join("..", ".."))
import random
import time

import sklearn
import numpy as np
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import pandas as pd

import Data
from signature_mahalanobis_knn import SignatureMahalanobisKNN, Mahalanobis

from signature_mahalanobis_knn.utils import compute_auc_given_dists
import UCR_data_support

In [2]:
DATA_DIR = "./data/"
# importlib.reload(Data)
# importlib.reload(SigMahaKNN)

In [3]:
n_neighbours = 50

In [4]:
data = Data.Data(n_samples=(800, 30, 30), if_sample=True)
data.load_pen_digit()

## Can we reproduce distances?

For depth 4, digit 0:

In [36]:
digit = 0
depth = 4
data.load_pen_digit(digit=digit)
print("doing digit", digit, "doing signature level ", depth)
signature_maha_knn = SignatureMahalanobisKNN()
# Measure the time spent on fit
start_time = time.time()
signature_maha_knn.fit(
    knn_library="sklearn",
    X_train=data.corpus,
    signature_kwargs={
        "augmentation_list": None,
        "depth": depth,
    },
)
fit_time = time.time() - start_time
print(f"fit_time: {fit_time}")

inlier_dists, inlier_ix = signature_maha_knn.conformance(
    data.test_inlier, n_neighbors=n_neighbours, debug=True
)
outlier_dists, outlier_ix = signature_maha_knn.conformance(
    data.test_outlier, n_neighbors=n_neighbours, debug=True
)

doing digit 0 doing signature level  4
fit_time: 1.7248878479003906


In [39]:
print("index of closest train point to test_point[0] =", inlier_ix[0, 0])
print("Reported conformance =", inlier_dists[0])
print("Reported conformance squared =", inlier_dists[0] ** 2)

index of closest train point to test_point[0] = 342
Reported conformance = 0.03967368329534612
Reported conformance squared = 0.0015740011462194255


In [40]:
# Compute distances as per the old method
test_sig = (
    signature_maha_knn.signature_transform.transform(data.test_inlier[0]).iloc[0].values
)
train_sig = signature_maha_knn.signatures_train[inlier_ix[0, 0]]
dist = signature_maha_knn.mahal_distance.distance(test_sig, train_sig)
print("Reported distance =", dist)

Reported distance = 0.0015739502850919962


In [41]:
inlier_dists[0] ** 2 - dist

5.086112742926606e-08

In [42]:
inlier_dists[0] - np.sqrt(dist)

6.409984413327519e-07